# Random Forests 

Random forests is a supervised learning algorithm that is comprised of decision trees which are created from randomly selected data samples. The algorithm gets prediciton from each tree and selects the best solution by votes. The prediction result with the most votes becomes the final prediction. 

__Ensemble learning__ (or "ensembling") is simply the process of combining several models to solve a prediction problem, with the goal of producing a combined model that is more accurate than any individual model. For __classification__ problems, the combination is often done by majority vote. For __regression__ problems, the combination is often done by taking an average of the predictions. 

One popular method is __bootstrap aggregration/bagging__ where we take a subset of the data and train a model on each subset. Then the subsets are allowed to simultaneously vote on the outcome. This increases predictive accuracy by reducing the variance, similar to how cross-validation reduces the variance associated with the test set approach (for estimating out-of-sample error) by splitting many times an averaging the results.

Rather than building muiltple models, __boosting__ uses the output of one model as an input into the next forming a a serial/daisy-chained process. 

The last category is __stacking__, which incorperates bagging and boosting. In the first phase, multiple models are trained in parallel. Then, those models are used as inputs into a final model to give a prediction. 

### Advantages
* Random forests is considered as a highly accurate and robust method because of the number of decision trees participating in the process.
* It does not suffer from the overfitting problem. The main reason is that it takes the average of all the predictions, which cancels out the biases.
* The algorithm can be used in both classification and regression problems.
* Random forests can also handle missing values by using median values to replace continuous variables, and computing the proximity-weighted average of missing values.

### Disadvantages 
* Random forests is slow in generating predictions because it has multiple decision trees. 
* The model is difficult to interpret compared to a decision tree, where you can easily make a decision by following the path in the tree.

### Finding important features 
Random forests also offer good feature selection indictors by showing relative importance of each feature in a prediction. It uses gini index to describe the explanatory power of a the variable. If the decrease of impurity is large after the binary split, then the variable is signigicant. 

## Model Example 
We will be building a model on the [Lending Club](https://www.lendingclub.com/info/download-data.action) 2015 dataset to predict the state of a loan given. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [3]:
# Import Data
yr2015 = pd.read_csv('/Users/tsawaengsri/Desktop/Data Science Courses/Datasets/LoanStats3d_securev1.csv',
                    skipinitialspace=True,
                    header=1,
                    skipfooter=2)


/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """


In [4]:
yr2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
421090,36371250,NaN,10000,10000,10000.0,36 months,11.99%,332.10,B,B5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421091,36441262,NaN,24000,24000,24000.0,36 months,11.99%,797.03,B,B5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421092,36271333,NaN,13000,13000,13000.0,60 months,15.99%,316.07,D,D2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421093,36490806,NaN,12000,12000,12000.0,60 months,19.99%,317.86,E,E3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421094,36271262,NaN,20000,20000,20000.0,36 months,11.99%,664.20,B,B5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


Looks like there are many rows with missing data, but that is ok since random forests can work with that. 

In [5]:
yr2015.dtypes

id                         int64
member_id                float64
loan_amnt                  int64
funded_amnt                int64
funded_amnt_inv          float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 150, dtype: object

Since there are 150 attributes in this dataset, let's start to determine our model features by exploring the categorical data first. 

## Data Cleaning

When selecting categorical variables for our model, we will use get_dummy function, which is memory intensive if there are many stinctive values. To reduce the complexity of our model, we will take a look at all our categorical variables and convert those with over 30 distinctive values to numeric values. 

In [6]:
categorical = yr2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

term
2
int_rate
111
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
2
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
55
next_pymnt_d
5
last_credit_pull_d
56
application_type
2
verification_status_joint
1
hardship_flag
2
hardship_type
1
hardship_reason
9
hardship_status
3
hardship_start_date
30
hardship_end_date
31
payment_plan_start_date
31
hardship_loan_status
4
debt_settlement_flag
2
debt_settlement_flag_date
48
settlement_status
3
settlement_date
51


There are a couple of columns, such as emp_title and revol_util that have more than a thousand distinctive values. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

In [7]:
# Convert ID and Interest Rate to numeric.
yr2015['id'] = pd.to_numeric(yr2015['id'], errors='coerce')
yr2015['int_rate'] = pd.to_numeric(yr2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
yr2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc','last_pymnt_d','last_credit_pull_d',
            'hardship_end_date','payment_plan_start_date','debt_settlement_flag_date'], 1, inplace=True)

In [8]:
pd.get_dummies(yr2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,settlement_date_Nov-2017,settlement_date_Nov-2018,settlement_date_Oct-2015,settlement_date_Oct-2016,settlement_date_Oct-2017,settlement_date_Oct-2018,settlement_date_Sep-2015,settlement_date_Sep-2016,settlement_date_Sep-2017,settlement_date_Sep-2018
0,68355089,NaN,24700,24700,24700.0,11.99,820.28,65000.0,16.06,1,...,0,0,0,0,0,0,0,0,0,0
1,66310712,NaN,35000,35000,35000.0,14.85,829.90,110000.0,17.06,0,...,0,0,0,0,0,0,0,0,0,0
2,68407277,NaN,3600,3600,3600.0,13.99,123.03,55000.0,5.91,0,...,0,0,0,0,0,0,0,0,0,0
3,68476807,NaN,10400,10400,10400.0,22.45,289.91,104433.0,25.37,1,...,0,0,0,0,0,0,0,0,0,0
4,68341763,NaN,20000,20000,20000.0,10.78,432.66,63000.0,10.78,0,...,0,0,0,0,0,0,0,0,0,0
5,68426831,NaN,11950,11950,11950.0,13.44,405.18,34000.0,10.20,0,...,0,0,0,0,0,0,0,0,0,0
6,68426545,NaN,16000,16000,16000.0,12.88,363.07,70000.0,26.40,0,...,0,0,0,0,0,0,0,0,0,0
7,68476668,NaN,20000,20000,20000.0,9.17,637.58,180000.0,14.67,0,...,0,0,0,0,0,0,0,0,0,0
8,68338832,NaN,1400,1400,1400.0,12.88,47.10,64000.0,34.95,0,...,0,0,0,0,0,0,0,0,0,0
9,66624733,NaN,18000,18000,18000.0,19.48,471.70,150000.0,9.39,0,...,0,0,0,0,0,0,0,0,0,0


## Iteration 1

We will run the random forest classifier with all numeric and some categorical variables that have distinctive values less than 30. 

In [9]:
# Instantiating the model

rfc = ensemble.RandomForestClassifier()

X = yr2015.drop('loan_status', 1)
X = pd.get_dummies(X)
# Dropping NA instead of imputing because data is probably rich enough
X = X.dropna(axis=1)
Y = yr2015['loan_status']

cross_val_score(rfc, X, Y, cv=10)

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/a

array([0.98895854, 0.99069193, 0.99038256, 0.99083353, 0.99126078,
       0.99130827, 0.9915691 , 0.99183034, 0.9914266 , 0.99083288])

The score cross validation reports is the accuracy of the tree. Here we're about 99% accurate.

However, we did not refine the model so there maybe a few potential problems. Let's try to trim down as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

## Iteration 2

Let's try to identify features with the most gini importance and use those variables as features. 


#### Train a Random Forest Classifer 

Here I will do the model fitting and feature selection altogether in one line of code.
* Firstly, I specify the random forest instance, indicating the number of trees.
* Then I use selectFromModel object from sklearn to automatically select the features.


In [10]:
rfc.fit(X,Y)

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

#### Identify and Select Most Important Features

In [11]:
rfc1_fi = rfc.feature_importances_
indicies = np.argsort(rfc1_fi)
feat_names = X.columns

In [12]:
# Function to print the name and gini importance of each feature
def feat_importance(feat_names, model):
    for feature in zip(feat_names, model.feature_importances_):
        print(feature)

In [13]:
feat_importance(feat_names, rfc)

('id', 0.0015592562324726728)
('loan_amnt', 0.009115535972715902)
('funded_amnt', 0.023042597328191947)
('funded_amnt_inv', 0.015802110282992946)
('int_rate', 0.013932027265023256)
('installment', 0.011782261453960632)
('annual_inc', 0.0013240136589608071)
('delinq_2yrs', 0.00037689722484635606)
('fico_range_low', 0.0010330515653301487)
('fico_range_high', 0.0009090107129673758)
('inq_last_6mths', 0.00038662668940846916)
('open_acc', 0.0008366720875389198)
('pub_rec', 0.0002758048735497289)
('revol_bal', 0.0014150039436242297)
('total_acc', 0.0012806578826012975)
('out_prncp', 0.047390259873066706)
('out_prncp_inv', 0.16418579996956492)
('total_pymnt', 0.02439300596737999)
('total_pymnt_inv', 0.01877710873030703)
('total_rec_prncp', 0.08430329107511239)
('total_rec_int', 0.01396680417543875)
('total_rec_late_fee', 0.004295902416415712)
('recoveries', 0.11869767182600224)
('collection_recovery_fee', 0.07045092038290537)
('last_pymnt_amnt', 0.06966393363455946)
('last_fico_range_high', 0

In [14]:
from sklearn.feature_selection import SelectFromModel
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.01
sfm = SelectFromModel(rfc, threshold=0.01)

# Train the selector
sfm.fit(X, Y)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=0.01)

In [15]:
# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(feat_names[feature_list_index])

loan_amnt
funded_amnt_inv
installment
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
recoveries
collection_recovery_fee
last_pymnt_amnt
last_fico_range_high
last_fico_range_low
term_ 36 months
term_ 60 months
next_pymnt_d_Aug-2019
debt_settlement_flag_N


Let's use the base features into the next model. 

#### Create A Data Subset With Only The Most Important Features

In [16]:
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important = sfm.transform(X)

#### Train A New Random Forest Classifier Using Only Most Important Features

In [17]:
# Create a new random forest classifier for the most important features

cross_val_score(rfc, X_important, Y, cv=10)

array([0.99159424, 0.99375505, 0.99430078, 0.99370696, 0.99401553,
       0.9938018 , 0.99415774, 0.99460897, 0.99406274, 0.99425274])

There wasn't much of a change from our first iteration. Let's try using only the top 5 columns. 

## Iteration 3

In [18]:
# From the top 5 features 
feature_cols = yr2015.loc[:,['funded_amnt','installment','out_prncp','out_prncp_inv',
'total_pymnt']]

In [19]:
X1 = pd.get_dummies(feature_cols)
X1 = X1.dropna(axis=1)
Y = Y

rfc1 = ensemble.RandomForestClassifier()

cross_val_score(rfc1, X1, Y, cv=10)

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/a

array([0.96131928, 0.97433158, 0.97202631, 0.97385419, 0.97268992,
       0.97036263, 0.9686038 , 0.96736885, 0.9684138 , 0.96665638])

Those scores are still relatively high. Let's try to combine the top 5 features with PCA. 

## Iteration 4

In [20]:
from sklearn.decomposition import PCA
# Combining top 5 features
pca = PCA(n_components=5)
X2 = pca.fit_transform(X)

In [21]:
cross_val_score(rfc, X2, Y, cv=10)

array([0.12691741, 0.6672128 , 0.62613094, 0.53094277, 0.54503788,
       0.56823957, 0.56002565, 0.55945567, 0.6219156 , 0.68938656])

These results are very low. Let's go back to dropping columns. 

## Iteration 5

In [22]:
# From the top 3 features 
feature_cols2 = yr2015.loc[:,['funded_amnt','installment','out_prncp']]

In [23]:
X3 = pd.get_dummies(feature_cols2)
X3 = X3.dropna(axis=1)
Y = Y

rfc1 = ensemble.RandomForestClassifier()

cross_val_score(rfc1, X3, Y, cv=10)

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/a

array([0.76121955, 0.79788194, 0.79838997, 0.79831394, 0.80130138,
       0.79669429, 0.79376351, 0.79335977, 0.79668464, 0.77139193])

These scores are too low. Let's try 4 columns. 

## Iteration 5

In [24]:
# From the top 4 features 
feature_cols = yr2015.loc[:,['funded_amnt','installment','out_prncp','out_prncp_inv']]

In [25]:
X4 = pd.get_dummies(feature_cols2)
X4 = X4.dropna(axis=1)
Y = Y

rfc1 = ensemble.RandomForestClassifier()

cross_val_score(rfc1, X4, Y, cv=10)

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tsawaengsri/a

array([0.75618559, 0.79878425, 0.79831873, 0.79772026, 0.80018523,
       0.79743048, 0.79504595, 0.79231482, 0.79682713, 0.76464721])

This doesn't look good either. 